#### Stuctured Output [Pydantic]

In [2]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()


model = init_chat_model(model="gpt-5-mini")
model

ChatOpenAI(profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'text_inputs': True, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x1246f47d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1253520c0>, root_client=<openai.OpenAI object at 0x106dc71a0>, root_async_client=<openai.AsyncOpenAI object at 0x12496b590>, model_name='gpt-5-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [ ]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="Release year")
    director: str = Field(description="Directed of the movie")    
    ratings: float = Field(description="The movie ratings out of 10")
    
model_with_structure = model.with_structured_output(Movie)    
model_with_structure

In [ ]:
response = model_with_structure.invoke("Provide details about a movie inception")
response

#### Message output with parsed structure

In [ ]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details"""
    title: str = Field(...,description="The title of the movie")
    year: int = Field(...,description="Release year")
    director: str = Field(...,description="Directed of the movie")    
    ratings: float = Field(...,description="The movie ratings out of 10")

movie_with_structure = model.with_structured_output(Movie, include_raw=True)
response = movie_with_structure.invoke("Provide details about the moview inception")
response

### Nested Structure

In [ ]:
from pydantic import BaseModel,Field

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str 
    year: int 
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Bidget in million USD")
    
model_with_structure = model.with_structured_output(MovieDetails)

response = movie_with_structure.invoke("Provide details about the moview inception")
response


### TypedDict

In [3]:
from typing_extensions import TypedDict, Annotated

class Movie(TypedDict):
    """A movie with details"""
    title: Annotated[str, ...,"The title of the movie"]
    year: Annotated[int,...,"Release year"]
    director: Annotated[str,...,"Directed of the movie"]    
    ratings: Annotated[float,...,"The movie ratings out of 10"]

with_type_dict = model.with_structured_output(Movie)
response  = with_type_dict.invoke("Tell me about the movie superman")
response

{'title': 'Superman',
 'year': 1978,
 'director': 'Richard Donner',
 'ratings': 7.3}

In [7]:
from typing_extensions import TypedDict, Annotated

class Actor(TypedDict):
    name: str
    role: str

class MovieDetails(TypedDict):
    title: str 
    year: int 
    cast: list[Actor]
    genres: list[str]
    budget: float 
    
type_dict_str = model.with_structured_output(MovieDetails)

response = type_dict_str.invoke("Provide details about the moview inception")
response

{'title': 'Inception',
 'year': 2010,
 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Dom Cobb'},
  {'name': 'Joseph Gordon-Levitt', 'role': 'Arthur'},
  {'name': 'Elliot Page', 'role': 'Ariadne'},
  {'name': 'Tom Hardy', 'role': 'Eames'},
  {'name': 'Ken Watanabe', 'role': 'Mr. Saito'},
  {'name': 'Marion Cotillard', 'role': 'Mal Cobb'},
  {'name': 'Cillian Murphy', 'role': 'Robert Fischer'},
  {'name': 'Michael Caine', 'role': 'Professor Stephen Miles'},
  {'name': 'Pete Postlethwaite', 'role': 'Maurice Fischer'},
  {'name': 'Lukas Haas', 'role': 'Nash'}],
 'genres': ['Science fiction', 'Action', 'Heist', 'Thriller'],
 'budget': 160000000}

#### Dataclasses

In [8]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.messages import HumanMessage

@dataclass
class Movie:
    """A movie with details"""
    title:str
    year: int
    director: str    
    ratings: float
    
agent = create_agent(model="gpt-5-mini", response_format=Movie)

response = agent.invoke({"messages": [HumanMessage("provide details about inception")]})
response

{'messages': [HumanMessage(content='provide details about inception', additional_kwargs={}, response_metadata={}, id='da8ded6a-4e0e-4473-91d8-d4f9c5f3264c'),
  AIMessage(content='{"title":"Inception","year":2010,"director":"Christopher Nolan","ratings":87} \n\n\n\n\n', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 803, 'prompt_tokens': 165, 'total_tokens': 968, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DBTJ79Dn7LImfBWnvqnsEGMFX1utI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c7d2d-23d9-7151-98fc-aab229b11f6d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 165, 'output_tokens': 803, '